# AI Shape Detector - Google Colab Training

This notebook trains a CNN model to detect basic geometric shapes (circle, square, triangle, rectangle) using GPU acceleration.

## Setup Instructions:
1. **Enable GPU**: Go to `Runtime` → `Change runtime type` → Select `T4 GPU` → Save
2. **Run all cells** in order (Ctrl+F9 or Runtime → Run all)
3. **Download trained model** at the end

---

## 1. Check GPU Availability

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ GPU not available. Go to Runtime → Change runtime type → Select GPU")

## 2. Clone Repository from GitHub

In [ ]:
# Clone your GitHub repository
!git clone https://github.com/RadwanSusan/AI-shape-detector-model.git
%cd AI-shape-detector-model

## 3. Install Dependencies

In [ ]:
!pip install -q -r requirements.txt
print("✓ Dependencies installed!")

## 4. Generate Synthetic Dataset

This will create 6,000 shape images (1,500 per class)

In [ ]:
%cd src
!python generate_data.py
print("\n✓ Dataset generated!")

## 5. Preview Sample Images

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
import glob

shapes = ['circle', 'square', 'triangle', 'rectangle']
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for idx, shape in enumerate(shapes):
    # Get first image file from the shape folder
    shape_dir = f'data/train/{shape}'
    image_files = glob.glob(f'{shape_dir}/*.png')

    if image_files:
        img_path = image_files[0]  # Use first available image
        img = Image.open(img_path)
        axes[idx].imshow(img)
        axes[idx].set_title(shape.upper(), fontsize=14, fontweight='bold')
        axes[idx].axis('off')
    else:
        print(f"No images found in {shape_dir}")

plt.tight_layout()
plt.show()

## 6. Train the Model

Training with GPU acceleration (should take 1-3 minutes for 20 epochs)

In [ ]:
# Train for 20 epochs with GPU
!python train.py
print("\n✓ Training complete!")

## 7. View Training History

In [ ]:
from IPython.display import Image as IPImage
display(IPImage(filename='results/training_history.png'))

## 8. Evaluate Model on Test Set

In [ ]:
!python evaluate.py

## 9. View Evaluation Results

In [ ]:
# Display confusion matrix
from IPython.display import Image as IPImage
print("Confusion Matrix:")
display(IPImage(filename='results/confusion_matrix.png'))

print("\nSample Predictions:")
display(IPImage(filename='results/sample_predictions.png'))

## 10. Test Predictions on Custom Images

In [ ]:
# Test on a few images from test set
!python predict.py --dir data/test/circle --save

# Display results
from IPython.display import Image as IPImage
display(IPImage(filename='results/predictions/batch_predictions.png'))

## 11. Upload Your Own Image to Test

In [ ]:
from google.colab import files
import os

# Upload an image
print("Upload an image with a shape (circle, square, triangle, or rectangle):")
uploaded = files.upload()

# Get the uploaded filename
if uploaded:
    filename = list(uploaded.keys())[0]
    print(f"\nTesting on: {filename}")

    # Make prediction
    !python predict.py --image {filename} --save

    # Display result
    from IPython.display import Image as IPImage
    result_path = f'results/prediction_{filename}'
    if os.path.exists(result_path):
        display(IPImage(filename=result_path))

## 12. Model Information

In [ ]:
import torch
from model import ShapeDetectorCNN

# Load model
checkpoint = torch.load('models/best_model.pth', map_location='cpu')
model = ShapeDetectorCNN(num_classes=4)
model.load_state_dict(checkpoint['model_state_dict'])

print("Model Architecture:")
print("="*60)
print(model)
print("\n" + "="*60)
print(f"Total Parameters: {model.count_parameters():,}")
print(f"Classes: {checkpoint['class_names']}")
print(f"Best Validation Accuracy: {checkpoint['val_acc']:.2f}%")

## 13. Download Trained Model

Download the trained model to use locally on your computer

In [ ]:
from google.colab import files
import shutil

# Create a zip with model and results
!zip -r model_and_results.zip models results

print("Downloading trained model and results...")
files.download('model_and_results.zip')
print("✓ Download complete!")

## Next Steps

### To use the model locally:
1. Extract the downloaded `model_and_results.zip`
2. Copy the `models/` folder to your local project
3. Run predictions: `python predict.py --image your_image.png`

### To improve the model:
- Generate more training data (increase `num_samples_per_class`)
- Train for more epochs
- Add data augmentation
- Try different model architectures
- Add more shape classes

### Resources:
- GitHub: https://github.com/RadwanSusan/AI-shape-detector-model
- PyTorch Docs: https://pytorch.org/docs/
- Colab Docs: https://colab.research.google.com/

---
**Happy Training! 🎓🚀**